# <span style="color:blue"> KMeans (aprendizado não supervisionado) </span>
 É utilizado para:
- segmentação do mercado;
- agrupamento de clientes em potencial
- agrupamento de caracteristicas de clientes 
- produtos mais similares
- agrupar perfis de clientes 

Abaixo, em azul, foram os usados nesta prática

**PARAMETROS**: KMeans(parametro)
- <span style="color:blue"> **n_clusters**: int, default = 8 </span>

- **n_init** (é o numero de vezes que o algoritmo KMeans irá correr com diferentes tipos de centróides): int, default = 10

- **max_iter** (numero máximo de iterações que o algoritmo faz em um unico processamento): init, default = 300

- **tol (tolerancia)**: float, default = 1e-4

- **verbose (mode verbosidade)**: int, default = 0

- <span style="color:blue">**random_state** :determina um numero de randomico para gerar o centroide </span>

Quando os parametros não são determinados, os valores default é que são utilizados.


**ATRIBUTOS:** KMeans.atributos
- <span style="color:blue">**cluster_centers_** : coordenadas do centro do cluster </span>

- **labels_**: rotulos/legendas de cada ponto

- <span style="color:blue">**inertia_**: a soma do quadrado das distancias das amostras mais pertas dos centroides do cluster </span>

- **n_iter**: numero de iterações

- **n_features_in**: número de recursos vistos durante o ajuste

- **feature_name_in**: nome dos recursos


**METODOS**: KMeans.metodos
- <span style="color:blue">**.fit(x,[y, sample_weight])**: calcula os clusters </span>

- <span style="color:blue">**.fit_prediction(x,[y, sample_weight])**: calcula os centróides</span>

- **.fit_transform(x,[y, sample_weight])**: calcula e transforma x em espaço de distancia

- **.get_feature_names_out(x,[y, sample_weight])**: nomes de recursos de saída para transformação

- **.get_parameters(x,[y, sample_weight])**: obtém os parametros para o algoritmo

- **.predict(x,[y, sample_weight])**: prever o agrupamento mais próximo ao qual cada amostra em 'x'pertence

- **.score(x,[y, sample_weight])**: oposto do valor de x

- **.set_params(x,[y, sample_weight])**: define os parametros

- **.transform(x)**: transforma 'x' em um espaço de distancia do cluster


## Quais são as bibliotecas necessárias ?
Se for necessário instalar um biblioteca o comando é: **!pip install [nome biblioteca]**

In [ ]:
import pandas as pd #biblioteca para manipulação e análise de dados
import numpy as np #biblioteca para trabalhar arrays
import plotly.express as px  #biblioteca para a visualização dos dados
import plotly. graph_objects as go #biblioteca para criar gráficos
from sklearn.preprocessing import StandardScaler #para padronização dos dados
from sklearn.cluster import KMeans

Agora faremos a importação do nosso arquivo.
Lembrete: 
1. As barras invertidas (\\) devem ser duplas, no "caminho" do arquivo;
2. Verifique qual é o tipo de arquivo. Para cada tipo, há um pd.read;
3. Em caso de arquivo .csv, verificar qual é o separador e encoding.

In [ ]:
base = pd.read_csv('base_iris.csv', sep = ';')

Agora vamos ver o dataframe e ver alguns comandos de visualização e estatistica.

In [ ]:
base.head() #mostra o df

In [ ]:
base.shape #mostra a quantidade de linhas e de colunas

In [ ]:
base['Class'].unique() #Através do nome da coluna no df, ele retorna os valores únicos.

In [ ]:
base.describe() #mostra as estatisticas dos dados

# PETALAS

# Realizando agrupamento pelas petalas

In [ ]:
x = base.iloc[:, [2,3]].values
""" .iloc é um localizador baseado no indexador de posição ':' significa que ele vai pegar todos os registros
'[2,3]' significa em quais colunas que ele deve buscar e .values retorna uma array
"""
x[:3]

# Normalizando os dados
Precisamos colocar os dados no mesmo padrão e na mesma escala, pois o Kmeans realiza os dados baseados na distancia.

--------------

### **class sklearn.preprocessing.StandardScaler(*, copy=True, with_mean=True, with_std=True)**

Padronize recursos removendo a média e dimensionando para a variação da unidade.

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

- fit_transform(X[, y]): Ajustar aos dados e transformá-los.

In [ ]:
normalizar = StandardScaler()
x = normalizar.fit_transform(x)
x[:3]

# Calculando o número de Cluster
Utilizaremos o metodo do cotovelo que nos auxilia no nº de cluster ideal.
Faremos isso utilizando o WCSS.
WCSS é a soma da distancia quadrada entre cada ponto e o centroide em um cluster

In [ ]:
wcss = [] #cria lista vazia
for i in range(1,11):
    kmeans = KMeans(n_clusters=i, random_state=0) #executa o KMeans
    kmeans.fit(x) #realiza o treinamento
    wcss.append(kmeans.inertia_) #adiciona na lista os valores de WCSS

In [ ]:
#Visualizando os valores de WCSS
for i in range (len(wcss)):
    print('Cluster:', i, '- Valor do Wcss', wcss[i])

# Criando o gráfico do cotovelo

In [ ]:
cotovelo = px.line(x= range(1,11), y= wcss)
cotovelo.show()

# No gráfico abaixo, nós temos que buscar o 'ultimo' cluster sem muita variação com o próximo.
# Nesse caso, a quantidade ideal de cluster para nosso treinamento é 3.
# Observe que a diferença entre x=3 e x=4 são minimas.

In [ ]:
# Sabendo qual é o nosso numero ideal de cluster, alteramos o n_clusters.
kmeans = KMeans(n_clusters=3, random_state=0) #executa o KMeans
label = kmeans.fit_predict(x) #calcular centros de cluster e prever o índice de cluster para cada amostra

In [ ]:
label

In [ ]:
centroides = kmeans.cluster_centers_ #calcula as coordenadas nos centroides.
centroides

# Gráfico de agrupamento

In [ ]:
petala = px.scatter( x= x[:,0], y = x[:,1], color = label)
center = px.scatter (x= centroides[:,0], y=centroides[:,1], size = [7,7,7])
grafico = go.Figure (data = petala.data + center.data)
grafico.show()

# SEPALAS

In [ ]:
s = base.iloc[:,[0,1]].values

In [ ]:
s[:3]

### Não precisa normalizar de novo

In [ ]:
# Cálculo de WCSS
wcss_sep = []
for i in range(1,11):
    kmeans_s = KMeans(n_clusters=i, random_state=0)
    kmeans_s.fit(s)
    wcss_sep.append(kmeans_s.inertia_)

In [ ]:
for i in range(len(wcss_sep)):
    print('Cluster:', i, 'valor do wcss:', wcss_sep[i])

In [ ]:
# Gráfico do cotovelo
cot = px.line(x= range(1,11), y = wcss_sep)
cot.show()

In [ ]:
# Recalcular o Kmeans
kmeans_s = KMeans(n_clusters=3, random_state=0)
label_s =kmeans_s.fit_predict(s)
label

In [ ]:
cent2 = kmeans_s.cluster_centers_
cent2

In [ ]:
sepala = px.scatter( x= s[:,0], y = s[:,1], color = label_s)
center2 = px.scatter (x= cent2[:,0], y=cent2[:,1], size = [7,7,7])
grafico = go.Figure (data = sepala.data + center2.data)
grafico.show()

Original disponibilizado por Prof Leandro Lessa https://github.com/ProfLeandroLessa/FDA-material-video-aulas

Implementos Lidiane Aureliano https://github.com/laurelianox